In [ ]:
import matplotlib.pyplot as plt
import numpy as np

```css
GaussianClassifier
 ├── fit(X, y)
 │     ├── 计算每类均值 μ_k
 │     ├── 计算共享协方差 Σ
 │     └── 计算先验 P(y=k)
 ├── predict(X)
 │     ├── 对每类计算判别函数 g_k(x)
 │     └── 返回最大 g_k(x) 的类别
 └── predict_proba(X)

In [ ]:

class GaussianClassifier:
    def __init__(self):
        self.mean = {} # 创建字典
        self.prior = {}
        self.cov = None
        self.cov_inv = None

    def fit(self, X, y):
        classes = np.unique(y) # 取所有的类别
        N, D = X.shape
        self.classes = classes # 如果需要在下面的函数中也使用

        # 1. 计算均值和先验
        for c in classes:
            Xc = X[y == c] # 取所有为C类的数据的特征向量
            self.mean[c] = np.mean(Xc, axis=0) # axis=0代表一列一列的处理数据.得到与样本特征向量维度一样的均值向量
            # 因为我们求的是特征在D维空间的几何中心，所以是对每一个特征维度求均值。所以是一列一列的。
            self.prior[c] = len(Xc) / N

        # 2. 计算协方差
        cov = np.zeros((D, D))
        for c in classes:
            Xc = X[y == c]
            mu = self.mean[c] # D维向量 (D,)
            diff = Xc - mu # (Nc,D) - (D,) 自动广播，先变成(,D) 再扩展到Nc维，实现减法
            cov += diff.T @ diff # 协方差矩阵是 D,D 所以转置

        cov /= N
        self.cov = cov
        self.cov_inv = np.linalg.inv(cov) # 矩阵的逆

    def _discriminant(self, x, c):
        mu = self.mean[c]
        prior = self.prior[c]

        term_1 = np.log(prior)
        term_2 = -0.5 * mu @ self.cov_inv @ mu 
        term_3 = x @ self.cov_inv @ mu # x 必须在左边，方便矩阵预测

        return term_2 + term_1 + term_3

    def predict(self, X):
        preds = []
        for x in X: # 二维数组，永远还一行一行迭代. 对多维数组迭代时，迭代的单位是“最外层维度的元素”
            scores = {c: self._discriminant(x, c) for c in self.classes} # 返回键值对 label 和 对应的判别函数值 
            preds.append(max(scores, key=scores.get)) # max() 返回字典中最大的 键 就是key
        return np.array(preds, dtype=np.int64)

>关于predict,思想就是找argmax.</br>
>这里用字典实现，拓展性更好</br>
>max(scores),返回了最大的key. max(要比较的元素, key = 决定比较依据的函数) 将比较key 变成了比较 scores.get


In [ ]:
# 两个类
mean0 = np.array([0, 0])
mean1 = np.array([3, 3])

cov0 = np.array([[1, 0.2], [0.2, 1]])
cov1 = np.array([[1, -0.3], [-0.3, 1]])

# 每类 200 个样本
X0 = np.random.multivariate_normal(mean0, cov0, 200)
X1 = np.random.multivariate_normal(mean1, cov1, 200)

X = np.vstack([X0, X1])
y = np.array([0]*200 + [1]*200)


In [ ]:
def plot_decision_boundary(model, X, y, steps=300):

    x_min, x_max = X[:,0].min() - 1, X[:,0].max() + 1
    y_min, y_max = X[:,1].min() - 1, X[:,1].max() + 1

    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, steps),
        np.linspace(y_min, y_max, steps)
    )
    grid = np.c_[xx.ravel(), yy.ravel()]

    Z = model.predict(grid).reshape(xx.shape)

    plt.contourf(xx, yy, Z, alpha=0.3, cmap="Paired")
    plt.scatter(X[:,0], X[:,1], c=y, cmap="Paired", edgecolors='k')

    plt.title("Gaussian Classifier (Shared Covariance)")
    plt.show()

In [ ]:
clf = GaussianClassifier()
clf.fit(X, y)

In [ ]:
plot_decision_boundary(clf, X, y)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse


def plot_gaussian_classifier(clf, X, y, ellipse_nsig=2, grid=True):
    classes = clf.classes
    #colors = ['blue', 'green']
    colors = ['#1f77b4', '#2ca02c']

    plt.figure(figsize=(7, 7))
    ax = plt.gca()
    ax.set_facecolor('black')

    # Scatter plot for each class
    for idx, c in enumerate(classes):
        Xc = X[y == c]
        plt.scatter(Xc[:, 0], Xc[:, 1], s=2, alpha=0.3, color=colors[idx])

    # Function to draw covariance ellipse
    def draw_ellipse(mean, cov, color):
        vals, vecs = np.linalg.eigh(cov)
        order = np.argsort(vals)[::-1]
        vals = vals[order]
        vecs = vecs[:, order]

        theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))

        width, height = 2 * ellipse_nsig * np.sqrt(vals)
        ellipse = Ellipse(xy=mean, width=width, height=height, angle=theta,
                          edgecolor=color, lw=2, facecolor='none')
        ax.add_patch(ellipse)

    # Draw ellipse for each class
    for idx, c in enumerate(classes):
        mean_c = clf.mean[c]
        Xc = X[y == c]
        diff = Xc - mean_c
        cov_c = (diff.T @ diff) / len(Xc)
        draw_ellipse(mean_c, cov_c, colors[idx])

    # Draw decision boundary (only for 2 classes)
    if len(classes) == 2:
        c1, c2 = classes
        mu1 = clf.mean[c1]
        mu2 = clf.mean[c2]

        w = clf.cov_inv @ (mu1 - mu2)
        b = -0.5 * (mu1 @ clf.cov_inv @ mu1 - mu2 @ clf.cov_inv @ mu2)

        xx = np.linspace(X[:, 0].min() - 1, X[:, 0].max() + 1, 300)
        yy = (-w[0] * xx - b) / w[1]
        plt.plot(xx, yy, color='white', lw=2)

    plt.xlim(np.min(X[:, 0]) - 1, np.max(X[:, 0]) + 1)
    plt.ylim(np.min(X[:, 1]) - 1, np.max(X[:, 1]) + 1)
    if grid:
        plt.grid(True)
    plt.title("Gaussian Classifier Visualization")
    plt.show()

In [ ]:
clf = GaussianClassifier()
clf.fit(X, y)

plot_gaussian_classifier(clf, X, y)

# 自测

In [ ]:
class gauss:
    def __init__(self):
        self.mean = {}
        self.prior = {}
        self.cov = None
        self.cov_inv = None

    def fit(self, X, y): #计算prior cov 和 最终得分
        N, D = X.shape
        self.classes = np.unique(y)

        # compute prior and mean
        for c in self.classes:
            Xc = X[y == c] # 取c类的样本出来
            self.mean[c] = np.mean(Xc, axis=0) #求特征中心. 字典的[key] = value  所以是映射赋值. 例如： 0： array[....]
            self.prior[c] = len(Xc) / N # len 总是返回第一维的大小，就是行，行就是样本数量

        #2. 计算样本的cov
        cov = np.zeros((D, D))
        for c in self.classes:
            Xc = X[y == c]
            mu = self.mean[c]
            diff = Xc - mu
            cov += diff.T @ diff

        cov /= N
        self.cov = cov
        self.cov_inv = np.linalg.inv(cov)

    def _discriminant(self, x, c): # 输入数据，和具体类别，进行计算
        mu = self.mean[c]
        prior = self.prior[c]

        term_1 = np.log(prior)
        term_2 = x @ self.cov_inv @ mu # x 必须在左边，方便矩阵预测
        term_3 = -0.5 * mu @ self.cov_inv @ mu

        return term_1 + term_2 + term_3

    def predict(self, X):
        preds = []
        for x in X: # 二维数组，永远还一行一行迭代. 对多维数组迭代时，迭代的单位是“最外层维度的元素”
            scores = {c: self._discriminant(x, c) for c in self.classes} # 对每个样本，遍历类别 从而计算
            preds.append(max(scores, key=scores.get))
        return np.array(preds)



In [ ]:
#